# einstein의 abstractive summarization 리뷰
[source](https://einstein.ai/research/your-tldr-by-an-ai-a-deep-reinforced-model-for-abstractive-summarization)

### 이상헌 From Voithru
 
### 필요한 두 가지 모델 : <br>
* A more contextual word generation model <br>
* A new way of training summarization models via reinforcement learning <br>

본문을 딥러닝 알고리즘에 넣고(encoding), 요약을 뽑아내는(decoding)이 모델은 어떻게 디자인 할 것인가? <br>
그리고 디자인한 모델을 어떻게 학습시킬 것이냐? <br>

요약이라는 task를 이 두 가지 로 세분화하여 설명한다. <br>

#### cf) summarization에 두 가지 방향성 :
* **Extractive summarization** <br>
 구절을 입력 데이터에서 찾아, 그 입력 데이터를 요약문에 붙이는 방식. <br>
 따라서 굉장히 robust하지만(여러 분야에 적용이 가능하지만), paraphrase가능치 않다.<br>
 
* **Abstractive summarization** <br>
 실제 '추상화된' 내용에서 요약을 뽑아낸다. <br>
 따라서 훨씬 더 어려운 것이다.<br>
 또한 실제 구현 단에서, 긴 output(long text output)을 낼 때 특히 관련없는 문장에 대한 에러가 많이 난다. <br>
 따라서 einstein은 이 문제를 타겟팅한 구현방식을 소개한다.
 
## model 1 :Reading and Generating model : <br>

가장 먼저 본 article이 소개하는 방식은 Reading 모델과 Generating 모델 모두 **RNN**을 차용하는 모델이다 <br>

당연히 RNN이 필연적으로 수반하는 *vanishing gradient problem* 때문에 잘 되지 않음을 강조하기 위해 본 모델을 먼저 소개하는 것이다. <br>

다만 괄목할만한 점은 Input과 output을 붙인 하나의 joint model을 만들 수 있는 점인데, <br>

Final hidden state of input 에 First hidden state of output을 붙여버린다. <br>

-> 이러한 형태를 **encoder-decoder RNN**, 혹은 **seq2seq**이라고 부른다. <br>

이 형태가 본 모델의 가장 기저에 깔려있다. <br>

![einstein.ai의 모델](http://i.imgur.com/g6xlgcH.png)

본 그림을 확인해보면 input RNN은 *bidirectional*이고, output의 경우는 단방향이다. <br>

이는 input data는 전후의 맥락을 전부 고려해야 하고, output의 경우는 그렇지 않으니 감정적으로 명백하다. <br>

### encoder-decoder RNN에 추가한 두 가지 요소 : 

* ### 'Temporal attention' : let decoder look back at parts of the input document 
**attention function**이라는 함수를 통하여 input data가 내보내는 hidden state 값에만 의존하는 것이 아니라, <br>
input의 일정 부분에 방점을 줄 수 있다. <br>


* ### Looking back at what decoder generated. 
decoder를 사용할 경우, vanishing gradient 문제로 인하여 같은 내용을 반복하는 상태가 나올 수 있다.<br>
이러한 상황을 대비하여 이전 hidden states를 decoder 자체가 볼 수 있도록하는 하나의 방식을 제공해야 한다.<br>
이건 가능케 하는 것이 **intra-decoder attention function** 이다.

![final model idea](http://i.imgur.com/fdrA7wp.png)

## model 2 : How to train the model?

#### cf) summarization에 두 가지 방향성 :
* **supervised learning** <br>
 training set에 output 값이 있는 경우. <br>
 이렇게 training 하면 다만 문제는 요약의 잘잘못을 떠나서 재미가 없다. <br>
 장난이고, 첫 몇 글자에 따라 다른 가치있는 summary가 나올 수 없다. <br>
 
 
* **Reinforcement learning** <br>
 1. summary를 모델이 뽑고, <br>
 2. scorer가 label과 비교하여 score를 낸다. <br>
 
 QQQQ supervised learning 에서는 local supervision signal을 각 word마다 트레이닝 시 사용하고,<br>
 RL에서는 이 대신 reward signal이라는, output 전체를 위한 score를 사용한다는데 이해가 잘 안갑니다. <br>
 
## RL에서, 어떤 score를 사용하는 건가요? - ROUGE!

** ROUGE = Recall-Oriented Understudy for Gisting Evaluation ** <br>
* 두 sub-pharse(어절)의 유사도를 판단하는 방식 <br>
* ROUGE-1, ROUGE-2, ROUGE-L 등은 다만 다른 길이를 비교하는 것이다. <br>

근데 제일 높은 ROUGE가 제일 좋은 summary냐? NO!!! <br>
요약이란건 사람마다 전혀 다르게 할 수 있지만, 그런 요약들이 전부 좋은 요약일 수 있다. <br>
그리고 실험 상, ROUGE가 제일 높은 summary는 읽을만한 요약이 아니라는 결과 또한 있다. <br>

그래서 제일 좋은건, **ROUGE**와 **supervised learning**을 둘 다 쓰는것! <br>
이렇게 해서 word level supervised learning은 생산된 문장이 문법적으로 오류가 없도록 잡아주고 <br>
ROUGE는 요약되어야만 하는 내용들이 들어있는가, 이것을 판단해주는 근거가 된다. <br>